In [19]:
import os
import pandas as pd
import numpy as np
import requests
import pandas as pd
from sqlalchemy import create_engine


In this section, we will ingest the training, test, and ground truth datasets from Azure storage. The training data comprises multiple multivariate time series, where each series is generated from a different engine of the same type. Each time series uses "cycle" as the time unit and includes 21 sensor readings per cycle.

The testing data follows the same schema as the training data, containing the same sensor readings and cycle information. However, the testing data does not indicate the cycle at which a failure occurs. To address this, we use the ground truth data, which provides the number of remaining operational cycles for the engines in the testing dataset.

For more detailed information about the types of data used in this notebook, please refer to the Predictive Maintenance dataset documentation.

In [20]:
urls = [
    "http://azuremlsamples.azureml.net/templatedata/PM_train.txt",
    "http://azuremlsamples.azureml.net/templatedata/PM_test.txt",
    "http://azuremlsamples.azureml.net/templatedata/PM_truth.txt"
]

for url in urls:
    filename = url.split("/")[-1]
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"{filename} downloaded successfully")

# Now you can load the downloaded datasets into pandas dataframes
train_df = pd.read_csv("PM_train.txt", sep=" ", header=None)
test_df = pd.read_csv("PM_test.txt", sep=" ", header=None)
truth_df = pd.read_csv("PM_truth.txt", sep=" ", header=None)

print("Training dataset:")
print(train_df.head())
print("Test dataset:")
print(test_df.head())
print("Truth dataset:")
print(truth_df.head())


PM_train.txt downloaded successfully
PM_test.txt downloaded successfully
PM_truth.txt downloaded successfully
Training dataset:
   0   1       2       3      4       5       6        7        8      9   \
0   1   1 -0.0007 -0.0004  100.0  518.67  641.82  1589.70  1400.60  14.62   
1   1   2  0.0019 -0.0003  100.0  518.67  642.15  1591.82  1403.14  14.62   
2   1   3 -0.0043  0.0003  100.0  518.67  642.35  1587.99  1404.20  14.62   
3   1   4  0.0007  0.0000  100.0  518.67  642.35  1582.79  1401.87  14.62   
4   1   5 -0.0019 -0.0002  100.0  518.67  642.37  1582.85  1406.22  14.62   

   ...       18      19    20   21    22     23     24       25  26  27  
0  ...  8138.62  8.4195  0.03  392  2388  100.0  39.06  23.4190 NaN NaN  
1  ...  8131.49  8.4318  0.03  392  2388  100.0  39.00  23.4236 NaN NaN  
2  ...  8133.23  8.4178  0.03  390  2388  100.0  38.95  23.3442 NaN NaN  
3  ...  8133.83  8.3682  0.03  392  2388  100.0  38.88  23.3739 NaN NaN  
4  ...  8133.80  8.4294  0.03  393  238

In [21]:
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

Addinnf into the training set the RUL

In [22]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


In [23]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)
# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)

test_df.drop("RUL",axis=1, inplace= True)

In [24]:
test_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130


Now we need to safe all the information in our local SQL database for having acces whenever we want

In [25]:
db_config = {
    'host': 'localhost',      # Host is 'localhost'
    'user': 'root',           # User is 'root'
    'password': 'durus1234',  # Replace with your actual MySQL password
    'database': 'tfm'         # Your database name is 'tfm'
}

In [26]:
engine = create_engine(f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

# Insert data into MySQL
try:
    # If the table already exists, you can set if_exists to 'append' or 'replace'
    test_df.to_sql('motor', con=engine, if_exists='replace', index=False)
    print("Data inserted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Data inserted successfully.


Now lets save the data locally for initiating the dat analysis

In [8]:
data_directory = 'C:/Users/daric/OneDrive/Desktop/TFM/tfm_predictive_maintenance/data'

# Save train_df to CSV
train_df.to_csv(f'{data_directory}/train_df.csv', index=False)

# Save test_df to CSV
test_df.to_csv(f'{data_directory}/test_df.csv', index=False)